In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_2/models/B5R2b5_5FP_1FC_fold2_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold2.csv') #Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4600, 25)
(4600, 25)
Normal:  (3005, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4600 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.09441096,  0.23918903, -0.20197257, ...,  0.22789359,
         0.51661396, -0.14124611],
       [ 0.16948406,  0.47768492,  0.14611943, ..., -0.13431966,
        -0.13310435,  0.04047935],
       [ 0.21317612,  0.34521919,  0.11112978, ..., -0.13038905,
        -0.22664231,  0.46842116],
       ...,
       [ 0.00440964, -0.19099697,  0.04486526, ...,  0.26192284,
         0.54269552,  0.37500393],
       [-0.15725851, -0.182146  ,  0.2596612 , ...,  0.17087623,
         0.51527709,  0.24497809],
       [-0.12479487, -0.20628045, -0.23066641, ...,  0.01210123,
        -0.23256479,  0.09433945]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.094411,0.239189,-0.201973,-0.001699,0.095731,0.005344,-0.189745,0.622467,0.073471,0.596786,...,-0.080699,-0.205078,0.146719,0.368106,-0.105726,0.582303,-0.147325,0.227894,0.516614,-0.141246
1,0.169484,0.477685,0.146119,0.018673,0.160418,-0.087399,0.631629,0.461510,-0.157241,0.221331,...,-0.123853,-0.204001,-0.202179,-0.111395,-0.172670,0.307085,0.389533,-0.134320,-0.133104,0.040479
2,0.213176,0.345219,0.111130,-0.011609,0.103672,-0.151683,0.866202,0.031669,-0.183181,0.145087,...,-0.161040,-0.125015,-0.218517,-0.171401,-0.211121,0.177539,0.581912,-0.130389,-0.226642,0.468421
3,-0.154059,-0.184366,-0.135882,0.006844,0.078435,-0.056761,-0.186001,0.306098,-0.091442,0.118542,...,0.201170,-0.019278,-0.089758,0.176530,-0.175952,-0.082821,-0.218357,0.152760,0.189012,-0.172847
4,0.084992,0.242587,-0.188342,0.220438,0.488157,0.292786,-0.166314,0.708275,0.278285,0.918924,...,0.312749,-0.218567,0.179008,0.530495,-0.213046,0.484237,-0.086485,0.053070,0.426111,-0.173378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,-0.200352,-0.041706,-0.152331,-0.098862,0.092539,-0.149784,-0.037442,0.332439,0.203684,0.623621,...,0.978651,-0.120094,0.909595,1.275295,-0.124503,0.252442,-0.044631,0.685414,0.519472,-0.120238
4596,-0.083680,-0.084362,-0.059828,0.162745,0.353058,0.121232,-0.132547,0.492538,0.141942,0.434728,...,0.136819,-0.067733,0.082504,0.119796,-0.199622,0.016825,-0.157459,-0.054615,0.176602,-0.121566
4597,0.004410,-0.190997,0.044865,0.456334,-0.209069,-0.196640,0.148205,0.173213,0.674409,-0.149127,...,0.232655,0.882290,0.120842,-0.035443,0.802990,-0.163906,-0.199476,0.261923,0.542696,0.375004
4598,-0.157259,-0.182146,0.259661,0.557378,-0.146514,-0.171299,-0.063838,0.225095,0.494478,-0.021032,...,0.493277,0.697589,0.322599,0.129194,0.529071,-0.224417,-0.208368,0.170876,0.515277,0.244978


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.094411,0.239189,-0.201973,-0.001699,0.095731,0.005344,-0.189745,0.622467,0.073471,0.596786,...,-0.105726,0.582303,-0.147325,0.227894,0.516614,-0.141246,Abnormal,P31,FP-B,AB01
1,0.169484,0.477685,0.146119,0.018673,0.160418,-0.087399,0.631629,0.461510,-0.157241,0.221331,...,-0.172670,0.307085,0.389533,-0.134320,-0.133104,0.040479,Abnormal,P1,FP-A,AB01
2,0.213176,0.345219,0.111130,-0.011609,0.103672,-0.151683,0.866202,0.031669,-0.183181,0.145087,...,-0.211121,0.177539,0.581912,-0.130389,-0.226642,0.468421,Abnormal,P2,FP-A,AB01
3,-0.154059,-0.184366,-0.135882,0.006844,0.078435,-0.056761,-0.186001,0.306098,-0.091442,0.118542,...,-0.175952,-0.082821,-0.218357,0.152760,0.189012,-0.172847,Abnormal,P42,FP-B,AB01
4,0.084992,0.242587,-0.188342,0.220438,0.488157,0.292786,-0.166314,0.708275,0.278285,0.918924,...,-0.213046,0.484237,-0.086485,0.053070,0.426111,-0.173378,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,-0.200352,-0.041706,-0.152331,-0.098862,0.092539,-0.149784,-0.037442,0.332439,0.203684,0.623621,...,-0.124503,0.252442,-0.044631,0.685414,0.519472,-0.120238,Normal,P32,FP-B,Normal
4596,-0.083680,-0.084362,-0.059828,0.162745,0.353058,0.121232,-0.132547,0.492538,0.141942,0.434728,...,-0.199622,0.016825,-0.157459,-0.054615,0.176602,-0.121566,Normal,P42,FP-B,Normal
4597,0.004410,-0.190997,0.044865,0.456334,-0.209069,-0.196640,0.148205,0.173213,0.674409,-0.149127,...,0.802990,-0.163906,-0.199476,0.261923,0.542696,0.375004,Normal,P52,FP-C,Normal
4598,-0.157259,-0.182146,0.259661,0.557378,-0.146514,-0.171299,-0.063838,0.225095,0.494478,-0.021032,...,0.529071,-0.224417,-0.208368,0.170876,0.515277,0.244978,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_5FP_RF_5FP_fold2_1.csv') #เปลี่ยนชื่อไฟล์